# Trajectories

> trajectories.

In [ ]:
#| default_exp named.trj

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os, copy, itertools
import numpy as np, pandas as pd

from dataclasses import dataclass, field
from rich.repr import auto as rich_auto, Result as RichReprResult

from typing import overload, Union, List, Tuple, Optional, Dict
from enum import Enum, auto

In [ ]:
from litds.named.arr import (
    NamedArray, NamedAxes, NamedAxis, NamedArrayDynamicAttrsMixin
)

In [ ]:
#| export
class TrajectoryDim(Enum):
    traj = auto()
    obsv = auto()
    feat = auto()

In [ ]:
#| export
class GeneTrajectoryDim(Enum):
    traj = auto()
    cell = auto()
    gene = auto()

'''
NOTE: originally the tuple had the followings strings and variable names:
- (timebins, cells, genes) 
- (tbins, cells, genes)
It is assumed that the trajectory matrix has three axes of assumed shape.
'''

In [ ]:
#| export
class ObservationTrajectories(NamedArray):
    DIMS = NamedAxes([
        NamedAxis(TrajectoryDim.obsv.name, 0), 
        NamedAxis(TrajectoryDim.traj.name, 1),
    ])

#| export
class FeatureTrajectories(NamedArray):
    DIMS = NamedAxes([
        NamedAxis(TrajectoryDim.feat.name, 0), 
        NamedAxis(TrajectoryDim.traj.name, 1),
    ])

In [ ]:
#| export
class CellularTrajectories(NamedArray):
    DIMS = NamedAxes([
        NamedAxis(GeneTrajectoryDim.cell.name, 0), 
        NamedAxis(GeneTrajectoryDim.traj.name, 1),
    ])

#| export
class ExpressionTrajectories(NamedArray):
    DIMS = NamedAxes([
        NamedAxis(GeneTrajectoryDim.gene.name, 0), 
        NamedAxis(GeneTrajectoryDim.traj.name, 1),
    ])

In [ ]:
#| export
class Trajectories(NamedArray):
    DIMS = NamedAxes([NamedAxis(dim.name, i) for i, dim in enumerate(TrajectoryDim)])
    
    def to_obsv_x_traj(self, aggr=np.mean):
        agg_arr = aggr(np.asarray(self.transpose(
            TrajectoryDim.obsv.name, TrajectoryDim.traj.name, TrajectoryDim.feat.name,            
        )), axis=2)
        return ObservationTrajectories(agg_arr)
        
    def to_feat_x_traj(self, aggr=np.mean):
        agg_arr = aggr(np.asarray(self.transpose(
            TrajectoryDim.feat.name, TrajectoryDim.traj.name, TrajectoryDim.obsv.name,            
        )), axis=2)
        return FeatureTrajectories(agg_arr)

In [ ]:
#| export
class GeneTrajectories(NamedArray):
    DIMS = NamedAxes([NamedAxis(dim.name, i) for i, dim in enumerate(GeneTrajectoryDim)])
    
    def __new__(cls, *args, **kwargs):
        obj = super().__new__(cls, *args, **kwargs)        
        return obj
        
    def to_cell_x_traj(self, aggr=np.mean):
        '''
        Transpose and aggregate trajectories matrix (timebins, cells, gene)
        to produce (cells, timebins)
        ''' 
        agg_arr = aggr(np.asarray(self.transpose(
            GeneTrajectoryDim.cell.name, GeneTrajectoryDim.traj.name, GeneTrajectoryDim.gene.name,
        )), axis=2)
        return CellularTrajectories(agg_arr)
    
    def to_gene_x_traj(self, aggr=np.mean):
        '''
        Transpose and aggregate trajectories matrix (timebins, cells, gene)
        to produce (genes, timebins)
        '''
        agg_arr = aggr(np.asarray(self.transpose(
            GeneTrajectoryDim.gene.name, GeneTrajectoryDim.traj.name, GeneTrajectoryDim.cell.name,            
        )), axis=2)
        return ExpressionTrajectories(agg_arr)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()